# Web-scraping via Selenium

In [ ]:
! pip install selenium requests pandas 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time

# Set up Selenium webdriver
driver = webdriver.Chrome()
url = "https://discomap.eea.europa.eu/app/CO2Cars/"  # Replace with the actual URL
driver.get(url)

# Create the header list
header = [
    "ID", "MS", "Mp", "VFN", "Mh", "Man", "MMS", "TAN", "T", "Va", "Ve", "Mk", "Cn",
    "Ct", "Cr", "M (kg)", "Mt", "Enedc (g/km)", "Ewltp (g/km)", "W (mm)", "At1 (mm)",
    "At2 (mm)", "Ft", "Fm", "Ec (cm3)", "Ep (KW)", "Z (Wh/km)", "IT", "Ernedc (g/km)",
    "Erwltp (g/km)", "De", "Vf", "R", "Year", "Status", "Version_file", "E (g/km)",
    "Er (g/km)", "Zr", "Dr", "Fc", "VIN"
]

# Create an empty DataFrame
df = pd.DataFrame(columns=header)

# Wait for the table to load
wait = WebDriverWait(driver, 30)

while True:
    table = wait.until(EC.presence_of_element_located((By.ID, "mainTable")))
    time.sleep(1)
    # Extract the table rows and columns
    rows = table.find_elements(By.TAG_NAME, "tr")
    time.sleep(1)
    # Extract data from each data row
    for row in rows:
        row_data = [cell.text for cell in row.find_elements(By.TAG_NAME, "td")]
        df.loc[len(df)] = row_data

    # Try to click the next page button
    try:
        pagination = driver.find_element(By.CLASS_NAME, "pagination")
        next_page_btn = pagination.find_element(By.XPATH, '//a[contains(text(), ">")]')
        next_page_btn.click()
    except NoSuchElementException:
        break  # Exit the loop if there are no more pages

# Save the DataFrame to a JSON file
df.to_json('data.json', orient='records')

# Close the webdriver
driver.quit()


# Data Scraping via SQL query

In [ ]:
import pandas as pd
import requests
from urllib.parse import quote
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

query = quote("SELECT * FROM [CO2Emission].[latest].[co2cars] WHERE year='2019'")
base_url = f"https://discodata.eea.europa.eu/sql?query={query}&p={{page}}&nrOfHits=100&mail=null&schema=null"
total_pages = 1197209

def fetch_page(page):
    url = base_url.format(page=page)
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    data = response.json()
    if 'results' not in data or not data['results']:
        return None
    results = pd.json_normalize(data['results'])
    return results

start_time = time.time()
results_list = []
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(fetch_page, page) for page in range(1, total_pages)]
    for i, future in enumerate(as_completed(futures)):
        result = future.result()
        if result is not None:
            results_list.append(result)
        pages_left = total_pages - i - 1
        print(f"Pages left: {pages_left}")

df = pd.concat(results_list)

# Save the DataFrame to a JSON file
df.to_json('data.json', orient='records')

end_time = time.time()
print(f"Time taken: {end_time - start_time:.2f} seconds")


## Data Verification

In [ ]:
! pip install pandas psutil

In [2]:
import pandas as pd
import psutil

memory = psutil.virtual_memory()
available_memory = memory.available
memory_per_chunk = available_memory * 0.25
chunksize = int(memory_per_chunk / (8 * 40)) # assuming 30 columns and 8 bytes per cell

df1 = pd.DataFrame()

for chunk in pd.read_csv("data.csv", low_memory=False, chunksize=chunksize):
    # remove extra spaces from column names
    chunk.columns = chunk.columns.str.strip()
    
    # remove extra spaces from values
    chunk = chunk.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    
    # process the chunk here
    df1 = pd.concat([df1, chunk], ignore_index=True)


In [5]:
df1

,ID,Country,VFN,Mp,Mh,Man,MMS,Tan,T,Va,...,IT,Ernedc (g/km),Erwltp (g/km),De,Vf,Status,year,Date of registration,Fuel consumption,Electric range (km)
0,280059,PT,NaN,NaN,VOLVO,VOLVO CAR CORPORATION,VOLVO CAR CORPORATION,e4*2001/116*0076*43,M,MV74,...,NaN,NaN,NaN,NaN,NaN,P,2017,NaN,NaN,NaN
1,280059,DK,NaN,SUZUKI POOL,SUZUKI MOTOR CORPORATION,SUZUKI MOTOR CORPORATION,SUZUKI MOTOR CORPORATION,e4*2007/46*1205*01,AZ,C83S,...,NaN,NaN,NaN,NaN,NaN,P,2018,NaN,NaN,NaN
2,280059,BE,NaN,RENAULT,RENAULT,RENAULT SAS,RENAULT S.A.S.,e2*2007/46*0457*13,AH,0BE2,...,NaN,NaN,NaN,NaN,NaN,P,2019,NaN,NaN,NaN
3,280060,UK,NaN,NaN,UNKNOWN,UNKNOWN,VAUXHALL,NaN,A-H/MONOCAB,BF11,...,NaN,NaN,NaN,NaN,NaN,P,2010,NaN,NaN,NaN
4,280060,GB,NaN,NaN,AUDI AG,NaN,AUDI,NaN,B8,LCAKAQ1,...,NaN,NaN,NaN,NaN,NaN,P,2011,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69473300,77551041,FR,IP-5GB____EAT84556-VR3-0,STELLANTIS,PSA,PSA AUTOMOBILES SA,NaN,e2*2007/46*0628*18,F,4,...,NaN,NaN,NaN,NaN,NaN,P,2022,2022-06-22,1.8,51.0
69473301,77551042,FR,IP-5GB____EAT84556-VR3-0,STELLANTIS,PSA,PSA AUTOMOBILES SA,NaN,e2*2007/46*0628*18,F,4,...,NaN,NaN,NaN,NaN,NaN,P,2022,2022-05-12,1.8,51.0
69473302,77551043,FR,IP-5GB____EAT84556-VR3-0,STELLANTIS,PSA,PSA AUTOMOBILES SA,NaN,e2*2007/46*0628*18,F,4,...,NaN,NaN,NaN,NaN,NaN,P,2022,2022-04-29,1.8,51.0
69473303,77551044,FR,IP-5GB____EAT84556-VR3-0,STELLANTIS,PSA,PSA AUTOMOBILES SA,NaN,e2*2007/46*0628*19,F,4,...,NaN,NaN,NaN,NaN,NaN,P,2022,2022-08-25,1.8,51.0


In [3]:
print(df1.columns)

Index(['ID', 'Country', 'VFN', 'Mp', 'Mh', 'Man', 'MMS', 'Tan', 'T', 'Va',
       'Ve', 'Mk', 'Cn', 'Ct', 'Cr', 'r', 'm (kg)', 'Mt', 'Enedc (g/km)',
       'Ewltp (g/km)', 'W (mm)', 'At1 (mm)', 'At2 (mm)', 'Ft', 'Fm',
       'ec (cm3)', 'ep (KW)', 'z (Wh/km)', 'IT', 'Ernedc (g/km)',
       'Erwltp (g/km)', 'De', 'Vf', 'Status', 'year', 'Date of registration',
       'Fuel consumption', 'Electric range (km)'],
      dtype='object')


In [4]:
unique_values = df1['Ft'].unique()
unique_values

array(['DIESEL', 'Petrol', 'petrol', 'diesel', 'Diesel', 'PETROL',
       'NG-BIOMETHANE', 'NG-biomethane', 'Petrol/Electric', 'Electric',
       'Hydrogen', 'LPG', 'electric', 'Petrol-electric',
       'petrol/electric', 'Diesel-electric', 'Petrol-Electric', 'ng',
       'lpg', 'ELECTRIC', 'E85', 'Diesel/Electric', 'ng-biomethane',
       'petrol-electric', nan, 'diesel-electric', 'Other', 'Biodiesel',
       'Diesel-Electric', 'e85', 'NG-Biomethane', 'diesel/electric',
       'hydrogen', 'NG', 'PETROL/ELECTRIC', 'PETROL-ELECTRIC', 'unknown',
       '', 'Petrol-Gas', 'DIESEL-ELECTRIC', 'BIODIESEL',
       'Diesel/electric', 'NG_biomethane', 'DIESEL/ELECTRIC', 'HYDROGEN',
       'OTHER', 'UNKNOWN'], dtype=object)

In [5]:
# Convert the 'Ft' column to lowercase
df1['Ft'] = df1['Ft'].str.lower()

# Get the frequency of each unique value in the 'Ft' column
value_counts = df1['Ft'].value_counts(dropna=False)

# Print the result
print(value_counts)

Ft
petrol             40140216
diesel             21547179
electric            3543970
petrol/electric     2601606
lpg                 1053991
ng                   213790
diesel/electric      156779
NaN                   63901
ng-biomethane         61494
e85                   60809
                      17107
petrol-electric        6375
hydrogen               2613
unknown                2186
diesel-electric        1181
biodiesel                94
other                    12
petrol-gas                1
ng_biomethane             1
Name: count, dtype: int64


In [7]:
unique_values = df1['Fuel consumption'].unique()
unique_values

array([   nan,  4.5  ,  6.1  ,  6.2  ,  6.7  ,  5.8  ,  5.4  ,  5.3  ,
        4.6  ,  4.7  ,  4.8  ,  5.   ,  7.3  ,  6.4  ,  6.5  ,  5.9  ,
        5.7  ,  4.9  ,  6.   ,  6.9  ,  5.1  ,  4.2  ,  9.7  ,  1.3  ,
        8.6  ,  6.3  ,  4.4  ,  5.2  ,  2.6  ,  5.5  ,  5.6  ,  6.6  ,
        8.9  ,  4.   ,  2.1  ,  8.4  ,  7.   ,  1.4  ,  9.5  ,  7.1  ,
        8.2  ,  1.2  ,  7.8  ,  8.1  ,  8.3  ,  7.4  ,  8.5  ,  4.3  ,
        7.5  ,  7.6  ,  7.2  ,  4.1  ,  3.9  , 10.8  ,  2.   , 10.4  ,
        1.9  ,  1.5  ,  7.7  ,  1.   ,  6.8  ,  1.7  ,  3.1  ,  1.1  ,
        9.3  ,  2.2  ,  1.8  ,  8.   ,  9.1  ,  7.9  , 13.2  ,  8.8  ,
        2.8  ,  9.2  ,  0.7  , 12.   , 10.7  ,  3.3  , 13.   ,  0.8  ,
        1.6  ,  9.8  ,  8.7  ,  2.5  ,  9.   ,  2.7  , 11.3  , 10.1  ,
        2.4  ,  0.9  , 11.   ,  9.6  , 11.6  ,  3.8  ,  7.92 ,  9.4  ,
       10.6  , 14.8  ,  3.4  , 10.3  , 10.2  , 12.1  , 11.1  ,  2.9  ,
       11.8  , 13.4  , 16.4  ,  9.9  , 10.   , 10.9  , 11.5  , 12.9  ,
      

In [8]:
# Get the frequency of each unique value in the 'Ft' column
value_counts = df1['Fuel consumption'].value_counts(dropna=False)

# Print the result
print(value_counts)

Fuel consumption
NaN       56349328
5.300       717857
5.200       639352
5.400       615552
4.900       567382
            ...   
4.767            1
5.436            1
77.000           1
5.459            1
17.000           1
Name: count, Length: 544, dtype: int64


In [3]:
result = df1.stack()[df1.stack() == 'FCA ITALY SPA']
result

153639    Mp     FCA ITALY SPA
          Man    FCA ITALY SPA
395952    Mp     FCA ITALY SPA
          Man    FCA ITALY SPA
496405    Mp     FCA ITALY SPA
                     ...      
14741550  Man    FCA ITALY SPA
          MMS    FCA ITALY SPA
15138914  Mp     FCA ITALY SPA
          Man    FCA ITALY SPA
          MMS    FCA ITALY SPA
Length: 2781, dtype: object

## Data Cleaning and Data Manipulation

### Complete Script

In [ ]:
! pip install pandas numpy psutil openpyxl xlsxwriter 

In [1]:
import pandas as pd
import numpy as np
import psutil
import math

# Define the chunk size
memory = psutil.virtual_memory()
available_memory = memory.available
memory_per_chunk = available_memory * 0.05
chunksize = int(memory_per_chunk / (8 * 38)) # assuming 30 columns and 8 bytes per cell

# Initialize an empty DataFrame to store the results
df = pd.DataFrame()

# Process the data in chunks
for chunk in pd.read_csv("data.csv", low_memory=False, chunksize=chunksize):
    # Remove extra spaces from column names and values
    chunk.columns = chunk.columns.str.strip()
    chunk = chunk.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    
    # Append the processed chunk to the result DataFrame
    df = pd.concat([df, chunk], ignore_index=True)

# Group the result DataFrame and aggregate
grouped = df.groupby(['Mk', 'Cn', 'Ft', 'Country', 'year'], as_index=False)

def aggregate(group):
    # Get the first row of the group
    first_row = group.iloc[0]
    
    # Average all numeric columns
    numeric_columns = group.select_dtypes(include='number').columns
    averaged = group[numeric_columns].mean()
    
    # Update the first row with the averaged values
    first_row[numeric_columns] = averaged
    
    # Fill missing values in non-numeric columns with the first non-null value from the group
    non_numeric_columns = group.select_dtypes(exclude='number').columns
    first_row[non_numeric_columns] = group[non_numeric_columns].fillna(method='ffill').iloc[0]
    
    # Update the ID column with a comma-separated list of all IDs in the group
    first_row['ID'] = ', '.join(group['ID'].astype(str))
    
    return first_row

df = grouped.apply(aggregate)

# Reorder the columns to their original order
df = df[chunk.columns]

print(f'data_updated length:',len(df))

# Create a new column fuel_name and rename the Ft values according to the list
fuel_name_map = {
    'petrol': 'Gasoline/Petrol',
    'diesel': 'On-Road Diesel Fuel',
    'lpg': 'LPG',
    'e85': 'E85 Ethanol/Gasoline',
    'nan': 'UNKNOWN',
    'ng': 'Natural gas',
    'ng-biomethane': 'Natural gas -Biomethane',
    'petrol/electric': 'Gasoline/Petrol + Electric',
    'petrol-gas': 'Gasoline/Petrol',
    'diesel/electric': 'On-Road Diesel Fuel + Electric',
    'petrol-electric': 'Gasoline/Petrol + Electric',
    'diesel-electric': 'On-Road Diesel Fuel + Electric',
    'hydrogen': 'Hydrogen',
    'electric': 'Electric',
    '': 'UNKNOWN',
    'unknown': 'UNKNOWN',
    'biodiesel': '100% Biodiesel',
    'other': 'Other',
}
df['fuel_name'] = df['Ft'].str.lower().map(fuel_name_map)

# Create a new column vehicle_name and merge the columns Mk and Cn as 'Mk (Cn)' formatted values
df['vehicle_name'] = df['Mk'] + " (" + df['Cn'] + ")"

# Create another column for new_ID with Country_Mk_Cn_Ft_year formatted values
df['Metrikflow_ID'] = df['Country'].str[:2] + "_" + df['Mk'].str[:2] + "_" + df['Cn'].str[:3] + "_" + df['Ft'].str[:2] + "_" + df['year'].astype(int).astype(str).str[-2:]

# Get a list of all column names
columns = df.columns.tolist()

# Remove 'new_ID' from the list of column names
columns.remove('Metrikflow_ID')

# Create a new list of column names with 'new_ID' as the first element
new_columns = ['Metrikflow_ID'] + columns

# Reorder the columns in the DataFrame
df = df[new_columns]

df['electric_capability'] = df['fuel_name'].apply(lambda x: 'TRUE' if x == 'Electric' else 'FALSE')

# Read the json file with the list of regions
regions_df = pd.read_json('general_regions.json')

# Create a mapping from region_code to region/regional_conditions
region_map = regions_df.set_index('region_code')['region/regional_conditions'].to_dict()

# Create a new column region/regional_conditions in df1 and rename the values according to the region_map
df['region/regional_conditions'] = df['Country'].str[:2].map(region_map)

# Create a new column electric_usage and set its value to the values from the z (Wh/km) column converted from Wh/km to kWh/km
df['electric_usage'] = df['z (Wh/km)'] / 1000

# Create a dictionary that maps fuel names to their corresponding biogenic fraction
biogenic_fraction_map = {
    'Jet Fuel': '0.00%',
    'Aviation Gasoline': '0.00%',
    'Gasoline/Petrol': '0.00%',
    'On-Road Diesel Fuel': '0.00%',
    'Residual Fuel Oil (3s 5 and 6)': '0.00%',
    'LPG': '0.00%',
    'CNG': '0.00%',
    'LNG': '0.00%',
    'Ethanol': '100.00%',
    '100% Biodiesel': '100.00%',
    'E85 Ethanol/Gasoline': '78.56%',
    'B20 Biodiesel/Diesel': '18.93%',
    'UNKNOWN': '0.00%',
    'Other': '0.00%',
    'Electric': '0.00%',
    'Hydrogen': '0.00%',
    'On-Road Diesel Fuel + Electric': '0.00%',
    'Gasoline/Petrol + Electric': '0.00%',
    'On-Road Diesel Fuel + Electric': '0.00%',
    'Natural gas -Biomethane': '0.00%',
    'Natural gas': '0.00%',
}

# Create a new column biogenic_fraction and set its value based on the fuel_name column
df['biogenic_fraction'] = df['fuel_name'].map(biogenic_fraction_map)

df['data_status'] = 'final'

# Add new columns data_provider(Carbon dioxide emission in kg/km (original value)) and link(Carbon dioxide emission in kg/km (original value)) to the DataFrame
df['data_provider(Carbon dioxide emission in kg/km (original value))'] = 'European Environment Agency'
df['link(Carbon dioxide emission in kg/km (original value))'] = 'https://www.eea.europa.eu/data-and-maps/data/co2-cars-emission-22'

# Define the number of bins to use
num_bins = 100

# Calculate the width of each bin
bin_width = (df['ec (cm3)'].max() - df['ec (cm3)'].min()) / num_bins

# Create a list to store the rows of the new DataFrame
rows = []

# Iterate over the bins
for i in range(num_bins):
    # Calculate the lower and upper bounds of the current bin
    lower_ec = df['ec (cm3)'].min() + i * bin_width
    higher_ec = lower_ec + bin_width
    
    # Filter df to only include rows where the ec column value is within the current bin
    mask = (df['ec (cm3)'] >= lower_ec) & (df['ec (cm3)'] < higher_ec)
    filtered_df = df[mask]
    
    # Filter filtered_df to only include rows where the Fuel consumption column is not null
    filtered_df = filtered_df[filtered_df['Fuel consumption'].notnull()]

    # Group the filtered DataFrame by fuel_name, year, and Country
    grouped = filtered_df.groupby(['fuel_name', 'Country'])
    
    # Iterate over the groups
    for (fuel_name, country), group in grouped:
        # Calculate the average fuel consumption for the current group
        avg_fuel_consumption = group['Fuel consumption'].mean()
        
        # Create a new row for the current group and append it to the list of rows
        row = {
            'lower ec (cm3)': lower_ec,
            'higher ec (cm3)': higher_ec,
            'fuel_name': fuel_name,
            'Country': country,
            'fuel consumption (L/100km)': avg_fuel_consumption,
            'fuel consumption (m3/km)': avg_fuel_consumption / 100
        }
        rows.append(row)

# Create a new DataFrame from the list of rows
df_fuel_consumption = pd.DataFrame(rows)

print(f'length of the dataframe to be written in the csv file:',len(df))

def fill_fuel_consumption(row):
    # Find the row in df_fuel_consumption where the value of 'ec (cm3)' in df is within the range of 'lower ec (cm3)' and 'higher ec (cm3)'
    mask = (row['ec (cm3)'] >= df_fuel_consumption['lower ec (cm3)']) & (row['ec (cm3)'] < df_fuel_consumption['higher ec (cm3)'])
    filtered_df = df_fuel_consumption[mask]
    
    # If a matching row is found, return the value from the 'fuel consumption (L/100km)' column
    if len(filtered_df) > 0:
        return filtered_df.iloc[0]['fuel consumption (L/100km)']
    else:
        return np.nan

# Fill NaN values in the 'Fuel consumption' column with the values from df_fuel_consumption
df['Fuel consumption'] = df['Fuel consumption'].fillna(df.apply(fill_fuel_consumption, axis=1))

def get_emission_value(row):
    if not pd.isna(row['Ewltp (g/km)']):
        return row['Ewltp (g/km)'] / 1000
    elif not pd.isna(row['Enedc (g/km)']):
        return row['Enedc (g/km)'] / 1000
    # elif not pd.isna(row['E']):
    #     return row['E'] / 1000
    else:
        return np.nan

df['Carbon dioxide emission in kg/km (original value)'] = df.apply(get_emission_value, axis=1)
df['Carbon dioxide emission in kg/m^3 (original value)'] = df['Carbon dioxide emission in kg/km (original value)'] * df['Fuel consumption']

print(f'data_cleaned_and_manipulated length:',len(df))

# Define a function to write a DataFrame to an Excel file and split it into multiple sheets if it exceeds a maximum number of rows
def write_df_to_excel(df, filename, sheet_name, max_rows_per_sheet=1048575):
    # Calculate the number of sheets needed
    num_sheets = math.ceil(len(df) / max_rows_per_sheet)
    
    # Create a writer object
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    
    # Write each chunk of rows to a separate sheet
    for i in range(num_sheets):
        start_row = i * max_rows_per_sheet
        end_row = min((i + 1) * max_rows_per_sheet, len(df))
        sheetname = f'{sheet_name} {i + 1}' if num_sheets > 1 else sheet_name
        df.iloc[start_row:end_row].to_excel(writer, sheet_name=sheetname, index=False)
    
    # Close the writer object
    writer.close()

# Write df to an Excel file named 'output.xlsx' and split it into multiple sheets if it exceeds 1048575 rows
write_df_to_excel(df, 'data_cleaned_and_manipulated.xlsx', 'Sheet1')

# Write df_fuel_consumption to a new sheet named 'Fuel Consumption' in the same Excel file
with pd.ExcelWriter('data_cleaned_and_manipulated.xlsx', engine='openpyxl', mode='a') as writer:
    df_fuel_consumption.to_excel(writer, sheet_name='Fuel Consumption', index=False)

# Save the DataFrame to a JSON file
df.to_json('data.json', orient='records')

### Complete Scripts in Separate Cells

In [ ]:
################################################################## Use this if you have enough memory ###############################################################################

In [2]:
import pandas as pd
import numpy as np
import psutil

# Define the chunk size
memory = psutil.virtual_memory()
available_memory = memory.available
memory_per_chunk = available_memory * 0.05
chunksize = int(memory_per_chunk / (8 * 38)) # assuming 30 columns and 8 bytes per cell

# Initialize an empty DataFrame to store the results
df1 = pd.DataFrame()

# Process the data in chunks
for chunk in pd.read_csv("data.csv", low_memory=False, chunksize=chunksize):
    # Remove extra spaces from column names and values
    chunk.columns = chunk.columns.str.strip()
    chunk = chunk.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    
    # Append the processed chunk to the result DataFrame
    df1 = pd.concat([df1, chunk], ignore_index=True)

In [3]:
# Group the result DataFrame and aggregate
grouped = df1.groupby(['Mk', 'Cn', 'Ft', 'Country', 'year'], as_index=False)

def aggregate(group):
    # Get the first row of the group
    first_row = group.iloc[0]
    
    # Average all numeric columns
    numeric_columns = group.select_dtypes(include='number').columns
    averaged = group[numeric_columns].mean()
    
    # Update the first row with the averaged values
    first_row[numeric_columns] = averaged
    
    # Fill missing values in non-numeric columns with the first non-null value from the group
    non_numeric_columns = group.select_dtypes(exclude='number').columns
    first_row[non_numeric_columns] = group[non_numeric_columns].fillna(method='ffill').iloc[0]
    
    # Update the ID column with a comma-separated list of all IDs in the group
    first_row['ID'] = ', '.join(group['ID'].astype(str))
    
    return first_row


df = grouped.apply(aggregate)

In [4]:
del(df1)
del(grouped)
del(available_memory)
del(chunk)
del(chunksize)
del(memory)
del(memory_per_chunk)

In [ ]:
################################################################ Use this if you don't have enough memory ###########################################################################

In [1]:
import pandas as pd
import numpy as np
import psutil

# Define the chunk size
memory = psutil.virtual_memory()
available_memory = memory.available
memory_per_chunk = available_memory * 0.05
chunksize = int(memory_per_chunk / (8 * 38)) # assuming 30 columns and 8 bytes per cell

# Initialize an empty DataFrame to store the results
df = pd.DataFrame()

# Process the data in chunks
for chunk in pd.read_csv("data.csv", low_memory=False, chunksize=chunksize):
    # Remove extra spaces from column names and values
    chunk.columns = chunk.columns.str.strip()
    chunk = chunk.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    
    # Append the processed chunk to the result DataFrame
    df = pd.concat([df, chunk], ignore_index=True)

In [ ]:
df

In [2]:
# Group the result DataFrame and aggregate
grouped = df.groupby(['Mk', 'Cn', 'Ft', 'Country', 'year'], as_index=False)

def aggregate(group):
    # Get the first row of the group
    first_row = group.iloc[0]
    
    # Average all numeric columns
    numeric_columns = group.select_dtypes(include='number').columns
    averaged = group[numeric_columns].mean()
    
    # Update the first row with the averaged values
    first_row[numeric_columns] = averaged
    
    # Fill missing values in non-numeric columns with the first non-null value from the group
    non_numeric_columns = group.select_dtypes(exclude='number').columns
    first_row[non_numeric_columns] = group[non_numeric_columns].fillna(method='ffill').iloc[0]
    
    # Update the ID column with a comma-separated list of all IDs in the group
    first_row['ID'] = ', '.join(group['ID'].astype(str))
    
    return first_row


df = grouped.apply(aggregate)

In [ ]:
################################################################################## Core Code ########################################################################################

In [5]:
# Reorder the columns to their original order
df = df[chunk.columns]

print(f'data_updated length:',len(df))

data_updated length: 420357


In [8]:
df

,ID,Country,VFN,Mp,Mh,Man,MMS,Tan,T,Va,...,IT,Ernedc (g/km),Erwltp (g/km),De,Vf,Status,year,Date of registration,Fuel consumption,Electric range (km)
0,"118451, 118457, 118458, 118459, 118462, 118471...",DE,NaN,BMW GROUP,BMW AG,BAYERISCHE MOTOREN WERKE AG,BAYER.MOT.WERKE-BMW,NaN,1K2,2S12,...,NaN,NaN,NaN,NaN,NaN,P,2015.0,NaN,NaN,NaN
1,"118450, 118460, 118461, 118463, 118464, 118465...",DE,NaN,BMW GROUP,BMW AG,BAYERISCHE MOTOREN WERKE AG,BAYER.MOT.WERKE-BMW,NaN,1K2,2P32,...,NaN,NaN,NaN,NaN,NaN,P,2015.0,NaN,NaN,NaN
2,"118467, 118468",DE,NaN,BMW GROUP,BMW AG,BAYERISCHE MOTOREN WERKE AG,BAYER.MOT.WERKE-BMW,NaN,1K4,1V91,...,NaN,NaN,NaN,NaN,NaN,P,2015.0,NaN,NaN,NaN
3,"114868, 114869, 114870, 114894",DE,NaN,BMW GROUP,BMW AG,BAYERISCHE MOTOREN WERKE AG,BAYER.MOT.WERKE-BMW,NaN,F20,1V91,...,NaN,NaN,NaN,NaN,NaN,P,2015.0,NaN,NaN,NaN
4,"120653, 120654, 118452, 118469, 118470, 114836...",DE,NaN,BMW GROUP,BMW AG,BAYERISCHE MOTOREN WERKE AG,BAYER.MOT.WERKE-BMW,NaN,2R11,,...,NaN,NaN,NaN,NaN,NaN,P,2015.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420352,"4996880, 4996897, 5012500, 5012501, 5012502, 5...",SK,NaN,VW GROUP PC,SKODA,SKODA AUTO AS,ŠKODA AUTO AS,e8*2007/46*0317*05,3T,AADFGAX01,...,NaN,2.002381,NaN,NaN,0.0,P,2019.0,NaN,NaN,NaN
420353,"9805374, 9806584, 9806585, 9806616, 9806617, 9...",HR,NaN,VW GROUP PC,SKODA,SKODA AUTO AS,SKODA AUTO as tr Vaclava Klementa 869 Mlada ...,e8*2007/46*0317*08,3T,ACDNUAX11,...,NaN,NaN,1.65,NaN,NaN,P,2019.0,NaN,NaN,NaN
420354,"4996473, 4996490, 4996914, 4977135, 4978470, 4...",SK,NaN,VW GROUP PC,SKODA,SKODA AUTO AS,ŠKODA AUTO AS,e8*2007/46*0317*08,3T,AADPCAX01,...,e8 19,1.694286,NaN,NaN,0.0,P,2019.0,NaN,NaN,NaN
420355,4931258,SK,NaN,VW GROUP PC,SKODA,SKODA AUTO AS,ŠKODA AUTO AS,e11*2007/46*0010*10,5L,ACCFHCX1,...,NaN,NaN,NaN,NaN,NaN,P,2019.0,NaN,NaN,NaN


In [10]:
# Create a new column fuel_name and rename the Ft values according to the list
fuel_name_map = {
    'petrol': 'Gasoline/Petrol',
    'diesel': 'On-Road Diesel Fuel',
    'lpg': 'LPG',
    'e85': 'E85 Ethanol/Gasoline',
    'nan': 'UNKNOWN',
    'ng': 'Natural gas',
    'ng-biomethane': 'Natural gas -Biomethane',
    'petrol/electric': 'Gasoline/Petrol + Electric',
    'petrol-gas': 'Gasoline/Petrol',
    'diesel/electric': 'On-Road Diesel Fuel + Electric',
    'petrol-electric': 'Gasoline/Petrol + Electric',
    'diesel-electric': 'On-Road Diesel Fuel + Electric',
    'hydrogen': 'Hydrogen',
    'electric': 'Electric',
    '': 'UNKNOWN',
    'unknown': 'UNKNOWN',
    'biodiesel': '100% Biodiesel',
    'other': 'Other',
}
df['fuel_name'] = df['Ft'].str.lower().map(fuel_name_map)

# Create a new column vehicle_name and merge the columns Mk and Cn as 'Mk (Cn)' formatted values
df['vehicle_name'] = df['Mk'] + " (" + df['Cn'] + ")"

In [11]:
# Create another column for new_ID with Country_Mk_Cn_Ft_year formatted values
df['Metrikflow_ID'] = df['Country'].str[:2] + "_" + df['Mk'].str[:2] + "_" + df['Cn'].str[:3] + "_" + df['Ft'].str[:2] + "_" + df['year'].astype(int).astype(str).str[-2:]

# Get a list of all column names
columns = df.columns.tolist()

# Remove 'new_ID' from the list of column names
columns.remove('Metrikflow_ID')

# Create a new list of column names with 'new_ID' as the first element
new_columns = ['Metrikflow_ID'] + columns

# Reorder the columns in the DataFrame
df = df[new_columns]

df['electric_capability'] = df['fuel_name'].apply(lambda x: 'TRUE' if x == 'Electric' else 'FALSE')

C:\Users\Akshay\AppData\Local\Temp\ipykernel_22244\1020882182.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['electric_capability'] = df['fuel_name'].apply(lambda x: 'TRUE' if x == 'Electric' else 'FALSE')


In [12]:
# Read the json file with the list of regions
regions_df = pd.read_json('general_regions.json')

# Create a mapping from region_code to region/regional_conditions
region_map = regions_df.set_index('region_code')['region/regional_conditions'].to_dict()

# Create a new column region/regional_conditions in df1 and rename the values according to the region_map
df['region/regional_conditions'] = df['Country'].str[:2].map(region_map)

# Create a new column electric_usage and set its value to the values from the z (Wh/km) column converted from Wh/km to kWh/km
df['electric_usage'] = df['z (Wh/km)'] / 1000

# Create a dictionary that maps fuel names to their corresponding biogenic fraction
biogenic_fraction_map = {
    'Jet Fuel': '0.00%',
    'Aviation Gasoline': '0.00%',
    'Gasoline/Petrol': '0.00%',
    'On-Road Diesel Fuel': '0.00%',
    'Residual Fuel Oil (3s 5 and 6)': '0.00%',
    'LPG': '0.00%',
    'CNG': '0.00%',
    'LNG': '0.00%',
    'Ethanol': '100.00%',
    '100% Biodiesel': '100.00%',
    'E85 Ethanol/Gasoline': '78.56%',
    'B20 Biodiesel/Diesel': '18.93%',
    'UNKNOWN': '0.00%',
    'Other': '0.00%',
    'Electric': '0.00%',
    'Hydrogen': '0.00%',
    'On-Road Diesel Fuel + Electric': '0.00%',
    'Gasoline/Petrol + Electric': '0.00%',
    'On-Road Diesel Fuel + Electric': '0.00%',
    'Natural gas -Biomethane': '0.00%',
    'Natural gas': '0.00%',
}

# Create a new column biogenic_fraction and set its value based on the fuel_name column
df['biogenic_fraction'] = df['fuel_name'].map(biogenic_fraction_map)

df['data_status'] = 'final'

# Add new columns data_provider(Carbon dioxide emission in kg/km (original value)) and link(Carbon dioxide emission in kg/km (original value)) to the DataFrame
df['data_provider(Carbon dioxide emission in kg/km (original value))'] = 'European Environment Agency'
df['link(Carbon dioxide emission in kg/km (original value))'] = 'https://www.eea.europa.eu/data-and-maps/data/co2-cars-emission-22'

C:\Users\Akshay\AppData\Local\Temp\ipykernel_22244\3663125608.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['region/regional_conditions'] = df['Country'].str[:2].map(region_map)
C:\Users\Akshay\AppData\Local\Temp\ipykernel_22244\3663125608.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['electric_usage'] = df['z (Wh/km)'] / 1000
C:\Users\Akshay\AppData\Local\Temp\ipykernel_22244\3663125608.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [15]:
# Define the number of bins to use
num_bins = 100

# Calculate the width of each bin
bin_width = (df['ec (cm3)'].max() - df['ec (cm3)'].min()) / num_bins

# Create a list to store the rows of the new DataFrame
rows = []

# Iterate over the bins
for i in range(num_bins):
    # Calculate the lower and upper bounds of the current bin
    lower_ec = df['ec (cm3)'].min() + i * bin_width
    higher_ec = lower_ec + bin_width
    
    # Filter df to only include rows where the ec column value is within the current bin
    mask = (df['ec (cm3)'] >= lower_ec) & (df['ec (cm3)'] < higher_ec)
    filtered_df = df[mask]
    
    # Filter filtered_df to only include rows where the Fuel consumption column is not null
    filtered_df = filtered_df[filtered_df['Fuel consumption'].notnull()]

    # Group the filtered DataFrame by fuel_name, year, and Country
    grouped = filtered_df.groupby(['fuel_name', 'Country'])
    
    # Iterate over the groups
    for (fuel_name, country), group in grouped:
        # Calculate the average fuel consumption for the current group
        avg_fuel_consumption = group['Fuel consumption'].mean()
        
        # Create a new row for the current group and append it to the list of rows
        row = {
            'lower ec (cm3)': lower_ec,
            'higher ec (cm3)': higher_ec,
            'fuel_name': fuel_name,
            'Country': country,
            'fuel consumption (L/100km)': avg_fuel_consumption,
            'fuel consumption (m3/km)': avg_fuel_consumption / 100
        }
        rows.append(row)

# Create a new DataFrame from the list of rows
df_fuel_consumption = pd.DataFrame(rows)

print(f'length of the dataframe to be written in the csv file:',len(df))

length of the dataframe to be written in the csv file: 420357


In [16]:
df_fuel_consumption

,lower ec (cm3),higher ec (cm3),fuel_name,Country,fuel consumption (L/100km),fuel consumption (m3/km)
0,586.74,670.56,Gasoline/Petrol,AT,4.800000,0.048000
1,586.74,670.56,Gasoline/Petrol,NL,4.840000,0.048400
2,586.74,670.56,Gasoline/Petrol,SE,4.814286,0.048143
3,838.20,922.02,Gasoline/Petrol,BE,5.280769,0.052808
4,838.20,922.02,Gasoline/Petrol,DE,6.000000,0.060000
...,...,...,...,...,...,...
1774,7962.90,8046.72,Gasoline/Petrol,BG,21.500000,0.215000
1775,7962.90,8046.72,Gasoline/Petrol,DE,22.825000,0.228250
1776,7962.90,8046.72,Gasoline/Petrol,FR,25.200000,0.252000
1777,7962.90,8046.72,Gasoline/Petrol,NL,25.190000,0.251900


In [17]:
def fill_fuel_consumption(row):
    # Find the row in df_fuel_consumption where the value of 'ec (cm3)' in df is within the range of 'lower ec (cm3)' and 'higher ec (cm3)'
    mask = (row['ec (cm3)'] >= df_fuel_consumption['lower ec (cm3)']) & (row['ec (cm3)'] < df_fuel_consumption['higher ec (cm3)'])
    filtered_df = df_fuel_consumption[mask]
    
    # If a matching row is found, return the value from the 'fuel consumption (L/100km)' column
    if len(filtered_df) > 0:
        return filtered_df.iloc[0]['fuel consumption (L/100km)']
    else:
        return np.nan

# Fill NaN values in the 'Fuel consumption' column with the values from df_fuel_consumption
df['Fuel consumption'] = df['Fuel consumption'].fillna(df.apply(fill_fuel_consumption, axis=1))

def get_emission_value(row):
    if not pd.isna(row['Ewltp (g/km)']):
        return row['Ewltp (g/km)'] / 1000
    elif not pd.isna(row['Enedc (g/km)']):
        return row['Enedc (g/km)'] / 1000
    # elif not pd.isna(row['E']):
    #     return row['E'] / 1000
    else:
        return np.nan

df['Carbon dioxide emission in kg/km (original value)'] = df.apply(get_emission_value, axis=1)
df['Carbon dioxide emission in kg/m^3 (original value)'] = df['Carbon dioxide emission in kg/km (original value)'] * df['Fuel consumption']

print(f'data_cleaned_and_manipulated length:',len(df))

C:\Users\Akshay\AppData\Local\Temp\ipykernel_22244\4274023847.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Fuel consumption'] = df['Fuel consumption'].fillna(df.apply(fill_fuel_consumption, axis=1))


data_cleaned_and_manipulated length: 420357


C:\Users\Akshay\AppData\Local\Temp\ipykernel_22244\4274023847.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Carbon dioxide emission in kg/km (original value)'] = df.apply(get_emission_value, axis=1)
C:\Users\Akshay\AppData\Local\Temp\ipykernel_22244\4274023847.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Carbon dioxide emission in kg/m^3 (original value)'] = df['Carbon dioxide emission in kg/km (original value)'] * df['Fuel consumption']


In [18]:
import math

# Define a function to write a DataFrame to an Excel file and split it into multiple sheets if it exceeds a maximum number of rows
def write_df_to_excel(df, filename, sheet_name, max_rows_per_sheet=1048575):
    # Calculate the number of sheets needed
    num_sheets = math.ceil(len(df) / max_rows_per_sheet)
    
    # Create a writer object
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    
    # Write each chunk of rows to a separate sheet
    for i in range(num_sheets):
        start_row = i * max_rows_per_sheet
        end_row = min((i + 1) * max_rows_per_sheet, len(df))
        sheetname = f'{sheet_name} {i + 1}' if num_sheets > 1 else sheet_name
        df.iloc[start_row:end_row].to_excel(writer, sheet_name=sheetname, index=False)
    
    # Close the writer object
    writer.close()

# Write df to an Excel file named 'output.xlsx' and split it into multiple sheets if it exceeds 1048575 rows
write_df_to_excel(df, 'data_cleaned_and_manipulated.xlsx', 'Sheet1')

# Write df_fuel_consumption to a new sheet named 'Fuel Consumption' in the same Excel file
with pd.ExcelWriter('data_cleaned_and_manipulated.xlsx', engine='openpyxl', mode='a') as writer:
    df_fuel_consumption.to_excel(writer, sheet_name='Fuel Consumption', index=False)

# Save the DataFrame to a JSON file
df.to_json('data.json', orient='records')

d:\Jobs\MetrikFlow\Data Analysis\CO2_emissions_scraper\.venv\Lib\site-packages\xlsxwriter\worksheet.py:1262: UserWarning: Ignoring URL 'https://www.eea.europa.eu/data-and-maps/data/co2-cars-emission-22' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn(


### Complete Script (For seperate years)

In [ ]:
import pandas as pd
import numpy as np
import math
import openpyxl

# Define a list of filenames
filenames = ['2018.csv', '2019.csv']

# Create a list to store the DataFrames
dfs = []

# Iterate over the filenames
for filename in filenames:
    # Read the current file into a DataFrame
    df = pd.read_csv(filename, low_memory=False)
    
    # Append the DataFrame to the list of DataFrames
    dfs.append(df)

# Concatenate the DataFrames into a single DataFrame
df1 = pd.concat(dfs, ignore_index=True)

# Create a new column fuel_name and rename the Ft values according to the list
fuel_name_map = {
    'petrol': 'Gasoline/Petrol',
    'diesel': 'On-Road Diesel Fuel',
    'lpg': 'LPG',
    'e85': 'E85 Ethanol/Gasoline',
    'nan': 'UNKNOWN',
    'ng': 'Natural gas',
    'ng-biomethane': 'Natural gas -Biomethane',
    'petrol/electric': 'Gasoline/Petrol + Electric',
    'diesel/electric': 'On-Road Diesel Fuel + Electric',
    'hydrogen': 'Hydrogen',
    'electric': 'Electric'
}
df1['fuel_name'] = df1['Ft'].str.lower().map(fuel_name_map)

# Create a new column vehicle_name and merge the columns Mk and Cn as 'Mk (Cn)' formatted values
df1['vehicle_name'] = df1['Mk'] + " (" + df1['Cn'] + ")"

print(f'data_updated length:',len(df1))

# Write df1 into an Excel file, splitting the data into multiple sheets if it exceeds the maximum number of rows allowed in a single sheet
max_rows_per_sheet = 1048574
num_rows = len(df1)
num_sheets = math.ceil(num_rows / max_rows_per_sheet)

# with pd.ExcelWriter('data_updated.xlsx') as writer:
#     for i in range(num_sheets):
#         start_row = i * max_rows_per_sheet
#         end_row = min((i + 1) * max_rows_per_sheet, num_rows)
#         df1.iloc[start_row:end_row].to_excel(writer, sheet_name=f'sheet_{i}', index=False)

# Group the DataFrame by Mk, Cn, Ft, MS, and year
grouped = df1.groupby(['Mk', 'Cn', 'Ft', 'Country', 'year'], as_index=False)

# Define a function to apply to each group
def aggregate(group):
    # Get the first row of the group
    first_row = group.iloc[0]
    
    # Average all numeric columns
    numeric_columns = group.select_dtypes(include='number').columns
    averaged = group[numeric_columns].mean()
    
    # Update the first row with the averaged values
    first_row[numeric_columns] = averaged
    
    # Fill missing values in non-numeric columns with the first non-null value from the group
    non_numeric_columns = group.select_dtypes(exclude='number').columns
    first_row[non_numeric_columns] = group[non_numeric_columns].fillna(method='ffill').iloc[0]
    
    # Update the ID column with a comma-separated list of all IDs in the group
    first_row['ID'] = ', '.join(group['ID'].astype(str))
    
    return first_row

# Apply the function to each group and create a new DataFrame
df = grouped.apply(aggregate)

del(df1)

# Create another column for new_ID with Country_Mk_Cn_Ft_year formatted values
df['Metrikflow_ID'] = df['Country'].str[:2] + "_" + df['Mk'].str[:2] + "_" + df['Cn'].str[:3] + "_" + df['Ft'].str[:2] + "_" + df['year'].astype(int).astype(str).str[-2:]

# Get a list of all column names
columns = df.columns.tolist()

# Remove 'new_ID' from the list of column names
columns.remove('Metrikflow_ID')

# Create a new list of column names with 'new_ID' as the first element
new_columns = ['Metrikflow_ID'] + columns

# Reorder the columns in the DataFrame
df = df[new_columns]

df['electric_capability'] = df['fuel_name'].apply(lambda x: 'TRUE' if x == 'Electric' else 'FALSE')

# Read the json file with the list of regions
regions_df = pd.read_json('general_regions.json')

# Create a mapping from region_code to region/regional_conditions
region_map = regions_df.set_index('region_code')['region/regional_conditions'].to_dict()

# Create a new column region/regional_conditions in df1 and rename the values according to the region_map
df['region/regional_conditions'] = df['Country'].str[:2].map(region_map)

# Create a new column electric_usage and set its value to the values from the z (Wh/km) column converted from Wh/km to kWh/km
df['electric_usage'] = df['z (Wh/km)'] / 1000

# Create a dictionary that maps fuel names to their corresponding biogenic fraction
biogenic_fraction_map = {
    'Jet Fuel': '0.00%',
    'Aviation Gasoline': '0.00%',
    'Gasoline/Petrol': '0.00%',
    'On-Road Diesel Fuel': '0.00%',
    'Residual Fuel Oil (3s 5 and 6)': '0.00%',
    'LPG': '0.00%',
    'CNG': '0.00%',
    'LNG': '0.00%',
    'Ethanol': '100.00%',
    '100% Biodiesel': '100.00%',
    'E85 Ethanol/Gasoline': '78.56%',
    'B20 Biodiesel/Diesel': '18.93%',
    'UNKNOWN': '0.00%',
    'Diesel': '0.00%'
}

# Create a new column biogenic_fraction and set its value based on the fuel_name column
df['biogenic_fraction'] = df['fuel_name'].map(biogenic_fraction_map)

df['data_status'] = 'final'

# Add new columns data_provider(Carbon dioxide emission in kg/km (original value)) and link(Carbon dioxide emission in kg/km (original value)) to the DataFrame
df['data_provider(Carbon dioxide emission in kg/km (original value))'] = 'European Environment Agency'
df['link(Carbon dioxide emission in kg/km (original value))'] = 'https://www.eea.europa.eu/data-and-maps/data/co2-cars-emission-22'

# Define the number of bins to use
num_bins = 20

# Calculate the width of each bin
bin_width = (df['ec (cm3)'].max() - df['ec (cm3)'].min()) / num_bins

# Create a list to store the rows of the new DataFrame
rows = []

# Iterate over the bins
for i in range(num_bins):
    # Calculate the lower and upper bounds of the current bin
    lower_ec = df['ec (cm3)'].min() + i * bin_width
    higher_ec = lower_ec + bin_width
    
    # Filter df to only include rows where the ec column value is within the current bin
    mask = (df['ec (cm3)'] >= lower_ec) & (df['ec (cm3)'] < higher_ec)
    filtered_df = df[mask]
    
    # Group the filtered DataFrame by fuel_name, year, and Country
    grouped = filtered_df.groupby(['fuel_name', 'year', 'Country'])
    
    # Iterate over the groups
    for (fuel_name, year, country), group in grouped:
        # Calculate the average fuel consumption for the current group
        avg_fuel_consumption = group['Fuel consumption'].mean()
        
        # Create a new row for the current group and append it to the list of rows
        row = {
            'lower ec (cm3)': lower_ec,
            'higher ec (cm3)': higher_ec,
            'fuel_name': fuel_name,
            'year': year,
            'Country': country,
            'fuel consumption (L/100km)': avg_fuel_consumption,
            'fuel consumption (m3/km)': avg_fuel_consumption / 100
        }
        rows.append(row)

# Create a new DataFrame from the list of rows
df_fuel_consumption = pd.DataFrame(rows)

def fill_fuel_consumption(row):
    # Find the row in df_fuel_consumption where the value of 'ec (cm3)' in df is within the range of 'lower ec (cm3)' and 'higher ec (cm3)'
    mask = (row['ec (cm3)'] >= df_fuel_consumption['lower ec (cm3)']) & (row['ec (cm3)'] < df_fuel_consumption['higher ec (cm3)'])
    filtered_df = df_fuel_consumption[mask]
    
    # If a matching row is found, return the value from the 'fuel consumption (L/100km)' column
    if len(filtered_df) > 0:
        return filtered_df.iloc[0]['fuel consumption (L/100km)']
    else:
        return np.nan

# Fill NaN values in the 'Fuel consumption' column with the values from df_fuel_consumption
df['Fuel consumption'] = df['Fuel consumption'].fillna(df.apply(fill_fuel_consumption, axis=1))

def get_emission_value(row):
    if not pd.isna(row['Ewltp (g/km)']):
        return row['Ewltp (g/km)'] / 1000
    elif not pd.isna(row['Enedc (g/km)']):
        return row['Enedc (g/km)'] / 1000
    # elif not pd.isna(row['E']):
    #     return row['E'] / 1000
    else:
        return np.nan

df['Carbon dioxide emission in kg/km (original value)'] = df.apply(get_emission_value, axis=1)
df['Carbon dioxide emission in kg/m^3 (original value)'] = df['Carbon dioxide emission in kg/km (original value)'] * df['Fuel consumption']

print(f'data_cleaned_and_manipulated length:',len(df))

# Define a function to write a DataFrame to an Excel file and split it into multiple sheets if it exceeds a maximum number of rows
def write_df_to_excel(df, filename, sheet_name, max_rows_per_sheet=1048575):
    # Calculate the number of sheets needed
    num_sheets = math.ceil(len(df) / max_rows_per_sheet)
    
    # Create a writer object
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    
    # Write each chunk of rows to a separate sheet
    for i in range(num_sheets):
        start_row = i * max_rows_per_sheet
        end_row = min((i + 1) * max_rows_per_sheet, len(df))
        sheetname = f'{sheet_name} {i + 1}' if num_sheets > 1 else sheet_name
        df.iloc[start_row:end_row].to_excel(writer, sheet_name=sheetname, index=False)
    
    # Close the writer object
    writer.close()

# Write df to an Excel file named 'output.xlsx' and split it into multiple sheets if it exceeds 1048575 rows
write_df_to_excel(df, 'data_cleaned_and_manipulated.xlsx', 'Sheet1')

# Write df_fuel_consumption to a new sheet named 'Fuel Consumption' in the same Excel file
with pd.ExcelWriter('data_cleaned_and_manipulated.xlsx', engine='openpyxl', mode='a') as writer:
    df_fuel_consumption.to_excel(writer, sheet_name='Fuel Consumption', index=False)

# Save the DataFrame to a JSON file
df.to_json('data.json', orient='records')

In [8]:
df

,Metrikflow_ID,ID,Country,VFN,Mp,Mh,Man,MMS,Tan,T,...,Electric range (km),fuel_name,vehicle_name,electric_capability,region/regional_conditions,electric_usage,biogenic_fraction,data_status,data_provider(Carbon dioxide emission in kg/km (original value)),link(Carbon dioxide emission in kg/km (original value))
0,BE_8_ZER_Pe_18,116599,BE,NaN,NaN,AA-IVA,AA-IVA,8,e6*eva2017*2105*00,01,...,NaN,Gasoline/Petrol,8 (ZEROUNO),FALSE,Belgium,NaN,0.00%,final,European Environment Agency,https://www.eea.europa.eu/data-and-maps/data/c...
1,GB_AB_124_Pe_18,"5028952,5019447,5116854,6161582,6161583,616158...",GB,NaN,FCA ITALY SPA,FIAT GROUP,FCA ITALY SPA,ABARTH,e3*2007/46*0474*00,NFM,...,NaN,Gasoline/Petrol,ABARTH (124 GT MULTIAIR),FALSE,United Kingdom,NaN,0.00%,final,European Environment Agency,https://www.eea.europa.eu/data-and-maps/data/c...
2,GB_AB_124_Pe_18,"6001883,5116557,5187085,5170265,5100520,511205...",GB,NaN,FCA ITALY SPA,FIAT GROUP,FCA ITALY SPA,ABARTH,e3*2007/46*0474*00,NFM,...,NaN,Gasoline/Petrol,ABARTH (124 GT MULTIAIR AUTO),FALSE,United Kingdom,NaN,0.00%,final,European Environment Agency,https://www.eea.europa.eu/data-and-maps/data/c...
3,SI_AB_124_Pe_18,"4593123,4593124,4593125",SI,NaN,FCA ITALY SPA,FIAT GROUP,FCA ITALY SPA,FCA ITALY SPA,E11*2007/46*3320*05,NF,...,NaN,Gasoline/Petrol,ABARTH (124 SPIDER / /),FALSE,Slovenia,NaN,0.00%,final,European Environment Agency,https://www.eea.europa.eu/data-and-maps/data/c...
4,SI_AB_124_Pe_18,4593127,SI,NaN,FCA ITALY SPA,FIAT GROUP,FCA ITALY SPA,FCA ITALY SPA,E11*2007/46*3320*05,NF,...,NaN,Gasoline/Petrol,ABARTH (124 SPIDER / / CA),FALSE,Slovenia,NaN,0.00%,final,European Environment Agency,https://www.eea.europa.eu/data-and-maps/data/c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34885,HU_Vo_XC9_Di_18,"14955329,14955330,14955331,14955332,14955333,1...",HU,NaN,NaN,Duplicate,VOLVO CAR CORPORATION,Volvo,e4*2007/46*0929*06,L,...,NaN,On-Road Diesel Fuel,Volvo (XC90),FALSE,Hungary,NaN,0.00%,final,European Environment Agency,https://www.eea.europa.eu/data-and-maps/data/c...
34886,HU_Vo_XC9_Pe_18,"14955344,14955663,14955664,14955665,14955844,1...",HU,NaN,NaN,VOLVO,VOLVO CAR CORPORATION,Volvo,e4*2007/46*0929*06,L,...,NaN,Gasoline/Petrol,Volvo (XC90),FALSE,Hungary,NaN,0.00%,final,European Environment Agency,https://www.eea.europa.eu/data-and-maps/data/c...
34887,HU_Vo_XC9_Pe_18,"14956655,14956656,14956657,14956658,14956659,1...",HU,NaN,NaN,VOLVO,VOLVO CAR CORPORATION,Volvo,e4*2007/46*0929*09,L,...,NaN,Gasoline/Petrol,Volvo (XC90 T8 Twin Engine),FALSE,Hungary,NaN,0.00%,final,European Environment Agency,https://www.eea.europa.eu/data-and-maps/data/c...
34888,HU_Vo_XC9_Pe_18,"14955220,14955221,14955223,14955224,14955225,1...",HU,NaN,NaN,VOLVO,VOLVO CAR CORPORATION,Volvo,e4*2007/46*0929*05,L,...,NaN,Gasoline/Petrol + Electric,Volvo (XC90 T8 Twin Engine),FALSE,Hungary,0.182,NaN,final,European Environment Agency,https://www.eea.europa.eu/data-and-maps/data/c...


In [12]:
df_fuel_consumption

,lower ec (cm3),higher ec (cm3),fuel_name,year,Country,fuel consumption (L/100km),fuel consumption (m3/km)
0,600.0,2078.6,E85 Ethanol/Gasoline,2018.0,DE,NaN,NaN
1,600.0,2078.6,E85 Ethanol/Gasoline,2018.0,FR,NaN,NaN
2,600.0,2078.6,E85 Ethanol/Gasoline,2018.0,LV,NaN,NaN
3,600.0,2078.6,E85 Ethanol/Gasoline,2018.0,SE,NaN,NaN
4,600.0,2078.6,Electric,2018.0,LT,NaN,NaN
...,...,...,...,...,...,...,...
385,6514.4,7993.0,Gasoline/Petrol,2018.0,PT,NaN,NaN
386,6514.4,7993.0,Gasoline/Petrol,2018.0,RO,NaN,NaN
387,6514.4,7993.0,Gasoline/Petrol,2018.0,SE,NaN,NaN
388,6514.4,7993.0,Gasoline/Petrol,2018.0,SI,NaN,NaN
